In [1]:
import matplotlib
import uproot, uproot3
import numpy
import awkward
import numba
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
import pandas as pd
from tqdm import trange
import ROOT
import os,sys
from array import array
import time

Welcome to JupyROOT 6.22/06


## Lumi

In [2]:
# CMS_lumi
#   Initiated by: Gautier Hamel de Monchenault (Saclay)
#   Translated in Python by: Joshua Hardenbrook (Princeton)
#   Updated by:   Dinko Ferencek (Rutgers)
#

cmsText     = "CMS";
cmsTextFont   = 62  

writeExtraText = True
extraText   = "Preliminary"
extraTextFont = 52 

lumiTextSize     = 0.4
lumiTextOffset   = 0.2

cmsTextSize      = 0.45
cmsTextOffset    = 0.1

relPosX    = 0.045
relPosY    = 0.035
relExtraDY = 1.2

extraOverCmsTextSize  = 0.76

lumi_13TeV = "20.1 fb^{-1}"
lumi_8TeV  = "19.7 fb^{-1}" 
lumi_7TeV  = "5.1 fb^{-1}"
lumi_sqrtS = "59.7 fb^{-1} (13 TeV)"

drawLogo      = False

def CMS_lumi(pad,  iPeriod,  iPosX ):
    outOfFrame    = False
    if(iPosX/10==0 ): outOfFrame = True

    alignY_=3
    alignX_=2
    if( iPosX/10==0 ): alignX_=1
    if( iPosX==0    ): alignY_=1
    if( iPosX/10==1 ): alignX_=1
    if( iPosX/10==2 ): alignX_=2
    if( iPosX/10==3 ): alignX_=3
    align_ = 10*alignX_ + alignY_

    H = pad.GetWh()
    W = pad.GetWw()
    l = pad.GetLeftMargin()
    t = pad.GetTopMargin()
    r = pad.GetRightMargin()
    b = pad.GetBottomMargin()
    e = 0.025

    pad.cd()

    lumiText = ""
    if( iPeriod==1 ):
        lumiText += lumi_7TeV
        lumiText += " (7 TeV)"
    elif ( iPeriod==2 ):
        lumiText += lumi_8TeV
        lumiText += " (8 TeV)"

    elif( iPeriod==3 ):      
        lumiText = lumi_8TeV 
        lumiText += " (8 TeV)"
        lumiText += " + "
        lumiText += lumi_7TeV
        lumiText += " (7 TeV)"
    elif ( iPeriod==4 ):
        lumiText += lumi_13TeV
        lumiText += " (13 TeV)"
    elif ( iPeriod==7 ):
        if( outOfFrame ):lumiText += "#scale[0.85]{"
        lumiText += lumi_13TeV 
        lumiText += " (13 TeV)"
        lumiText += " + "
        lumiText += lumi_8TeV 
        lumiText += " (8 TeV)"
        lumiText += " + "
        lumiText += lumi_7TeV
        lumiText += " (7 TeV)"
        if( outOfFrame): lumiText += "}"
    elif ( iPeriod==12 ):
        lumiText += "8 TeV"
    elif ( iPeriod==0 ):
        lumiText += lumi_sqrtS
            
    print (lumiText)

    latex = ROOT.TLatex()
    latex.SetNDC()
    latex.SetTextAngle(0)
    latex.SetTextColor(ROOT.kBlack)    
    
    extraTextSize = extraOverCmsTextSize*cmsTextSize
    
    latex.SetTextFont(42)
    latex.SetTextAlign(31) 
    latex.SetTextSize(lumiTextSize*t)    

    latex.DrawLatex(1-r,1-t+lumiTextOffset*t,lumiText)

    if( outOfFrame ):
        latex.SetTextFont(cmsTextFont)
        latex.SetTextAlign(11) 
        latex.SetTextSize(cmsTextSize*t)    
        latex.DrawLatex(l,1-t+lumiTextOffset*t,cmsText)
  
    pad.cd()

    posX_ = 0
    if( iPosX%10<=1 ):
        posX_ =   l + relPosX*(1-l-r)
    elif( iPosX%10==2 ):
        posX_ =  l + 0.5*(1-l-r)
    elif( iPosX%10==3 ):
        posX_ =  1-r - relPosX*(1-l-r)

    posY_ = 1-t - relPosY*(1-t-b)

    if( not outOfFrame ):
        if( drawLogo ):
            posX_ =   l + 0.045*(1-l-r)*W/H
            posY_ = 1-t - 0.045*(1-t-b)
            xl_0 = posX_
            yl_0 = posY_ - 0.15
            xl_1 = posX_ + 0.15*H/W
            yl_1 = posY_
            CMS_logo = ROOT.TASImage("CMS-BW-label.png")
            pad_logo =  ROOT.TPad("logo","logo", xl_0, yl_0, xl_1, yl_1 )
            pad_logo.Draw()
            pad_logo.cd()
            CMS_logo.Draw("X")
            pad_logo.Modified()
            pad.cd()          
        else:
            latex.SetTextFont(cmsTextFont)
            latex.SetTextSize(cmsTextSize*t)
            latex.SetTextAlign(align_)
            latex.DrawLatex(posX_, posY_, cmsText)
            if( writeExtraText ) :
                latex.SetTextFont(extraTextFont)
                latex.SetTextAlign(align_)
                latex.SetTextSize(extraTextSize*t)
                latex.DrawLatex(posX_, posY_- relExtraDY*cmsTextSize*t, extraText)
    elif( writeExtraText ):
        if( iPosX==0):
            posX_ =   l +  relPosX*(1-l-r)
            posY_ =   1-t+lumiTextOffset*t

        latex.SetTextFont(extraTextFont)
        latex.SetTextSize(extraTextSize*t)
        latex.SetTextAlign(align_)
        latex.DrawLatex(posX_*1.20, posY_, extraText)      

    pad.Update()

## tdrstyle

In [3]:
tdrStyle =  ROOT.TStyle("","")

#for the canvas:
tdrStyle.SetCanvasBorderMode(0)
tdrStyle.SetCanvasColor(ROOT.kWhite)
tdrStyle.SetCanvasDefH(1000) #Height of canvas
tdrStyle.SetCanvasDefW(800) #Width of canvas
tdrStyle.SetCanvasDefX(0)   #POsition on screen
tdrStyle.SetCanvasDefY(0)


tdrStyle.SetPadBorderMode(0)
#tdrStyle.SetPadBorderSize(Width_t size = 1)
tdrStyle.SetPadColor(ROOT.kWhite)
tdrStyle.SetPadGridX(False)
tdrStyle.SetPadGridY(False)
tdrStyle.SetGridColor(0)
tdrStyle.SetGridStyle(3)
tdrStyle.SetGridWidth(1)

#For the frame:
tdrStyle.SetFrameBorderMode(1)
tdrStyle.SetFrameBorderSize(1)
tdrStyle.SetFrameFillColor(0)
tdrStyle.SetFrameFillStyle(0)
tdrStyle.SetFrameLineColor(1)
tdrStyle.SetFrameLineStyle(1)
tdrStyle.SetFrameLineWidth(1)

#For the histo:
#tdrStyle.SetHistFillColor(1)
#tdrStyle.SetHistFillStyle(0)
tdrStyle.SetHistLineColor(1)
tdrStyle.SetHistLineStyle(0)
tdrStyle.SetHistLineWidth(1)
#tdrStyle.SetLegoInnerR(Float_t rad = 0.5)
#tdrStyle.SetNumberContours(Int_t number = 20)

tdrStyle.SetEndErrorSize(2)
#tdrStyle.SetErrorMarker(20)
#tdrStyle.SetErrorX(0.)

tdrStyle.SetMarkerStyle(20)

#For the fit/function:
tdrStyle.SetOptFit(1)
tdrStyle.SetFitFormat("5.4g")
tdrStyle.SetFuncColor(2)
tdrStyle.SetFuncStyle(1)
tdrStyle.SetFuncWidth(1)

#For the date:
tdrStyle.SetOptDate(0)
# tdrStyle.SetDateX(Float_t x = 0.01)
# tdrStyle.SetDateY(Float_t y = 0.01)

# For the statistics box:
tdrStyle.SetOptFile(0)
tdrStyle.SetOptStat(0) # To display the mean and RMS:   SetOptStat("mr")
tdrStyle.SetStatColor(ROOT.kWhite)
tdrStyle.SetStatFont(42)
tdrStyle.SetStatFontSize(0.025)
tdrStyle.SetStatTextColor(1)
tdrStyle.SetStatFormat("6.4g")
tdrStyle.SetStatBorderSize(1)
tdrStyle.SetStatH(0.1)
tdrStyle.SetStatW(0.15)
# tdrStyle.SetStatStyle(Style_t style = 1001)
# tdrStyle.SetStatX(Float_t x = 0)
# tdrStyle.SetStatY(Float_t y = 0)

# Margins:
tdrStyle.SetPadTopMargin(0.05)
tdrStyle.SetPadBottomMargin(0.13)
tdrStyle.SetPadLeftMargin(0.13)
tdrStyle.SetPadRightMargin(0.02)

# For the Global title:

tdrStyle.SetOptTitle(0)
tdrStyle.SetTitleFont(42)
tdrStyle.SetTitleColor(1)
tdrStyle.SetTitleTextColor(1)
tdrStyle.SetTitleFillColor(10)
tdrStyle.SetTitleFontSize(0.05)
# tdrStyle.SetTitleH(0) # Set the height of the title box
# tdrStyle.SetTitleW(0) # Set the width of the title box
# tdrStyle.SetTitleX(0) # Set the position of the title box
# tdrStyle.SetTitleY(0.985) # Set the position of the title box
# tdrStyle.SetTitleStyle(Style_t style = 1001)
# tdrStyle.SetTitleBorderSize(2)

# For the axis titles:

tdrStyle.SetTitleColor(1, "XYZ")
tdrStyle.SetTitleFont(42, "XYZ")
tdrStyle.SetTitleSize(0.04, "XYZ")
#   tdrStyle.SetTitleXSize(Float_t size = 0.02) # Another way to set the size?
#   tdrStyle.SetTitleYSize(Float_t size = 0.02)
tdrStyle.SetTitleXOffset(0.9)
tdrStyle.SetTitleYOffset(1.25)
# tdrStyle.SetTitleOffset(1.1, "Y") # Another way to set the Offset

# For the axis labels:

tdrStyle.SetLabelColor(1, "XYZ")
tdrStyle.SetLabelFont(42, "XYZ")
tdrStyle.SetLabelOffset(0.007, "XYZ")
tdrStyle.SetLabelSize(0.03, "XYZ")

# For the axis:

tdrStyle.SetAxisColor(1, "XYZ")
tdrStyle.SetStripDecimals(True)
tdrStyle.SetTickLength(0.03, "XYZ")
tdrStyle.SetNdivisions(510, "XYZ")
tdrStyle.SetPadTickX(1)  # To get tick marks on the opposite side of the frame
tdrStyle.SetPadTickY(1)

# Change for log plots:
tdrStyle.SetOptLogx(0)
tdrStyle.SetOptLogy(0)
tdrStyle.SetOptLogz(0)

# Postscript options:
# tdrStyle.SetPaperSize(20.,20.)
# tdrStyle.SetLineScalePS(Float_t scale = 3)
# tdrStyle.SetLineStyleString(Int_t i, const char* text)
# tdrStyle.SetHeaderPS(const char* header)
# tdrStyle.SetTitlePS(const char* pstitle)

# tdrStyle.SetBarOffset(Float_t baroff = 0.5)
# tdrStyle.SetBarWidth(Float_t barwidth = 0.5)
# tdrStyle.SetPaintTextFormat(const char* format = "g")
# tdrStyle.SetPalette(Int_t ncolors = 0, Int_t* colors = 0)
# tdrStyle.SetTimeOffset(Double_t toffset)
# tdrStyle.SetHistMinimumZero(kTRUE)

tdrStyle.SetHatchesLineWidth(1)
tdrStyle.SetHatchesSpacing(0.5)

tdrStyle.cd()

## ratio function

In [4]:
from ROOT import TCanvas, TColor, TGaxis, TH1F, TPad
from ROOT import kBlack, kBlue, kRed

def createRatio(h1, h2):
    h3 = h1.Clone("h3")
    h3.SetLineColor(kBlack)
    h3.SetMarkerStyle(21)
    h3.SetTitle("")
    h3.SetMinimum(0.80)
    h3.SetMaximum(1.53)
    # Set up plot for markers and errors
    h3.Sumw2()
    h3.SetStats(0)
    h3.Divide(h2)

    # Adjust y-axis settings
    y = h3.GetYaxis()
    y.SetTitle("Data / MC ")
    y.SetNdivisions(105)
    y.SetTitleSize(20)
    y.SetTitleFont(43)
    y.SetTitleOffset(1.55)
    y.SetLabelFont(43)
    y.SetLabelSize(20)

    # Adjust x-axis settings
    x = h3.GetXaxis()
    x.SetTitleSize(20)
    x.SetTitleFont(43)
    x.SetTitleOffset(4.0)
    x.SetLabelFont(43)
    x.SetLabelSize(20)

    return h3


## Add Hist 

In [25]:
def AddHist(file, hist, isData, xsec, lumi, channel, branch):
    
    init_time = time.time()
    init_branches = ['channel_mark','HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL','HLT_Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL','HLT_Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ','HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8','HLT_Ele32_WPTight_Gsf','HLT_IsoMu24']
    
    if isData:
        print('is Data')
        init_branches.append(branch)
    else:
        print('is MC')
        add_branches = ['Generator_weight']
        gen_lepton_branches = uproot.open(file+':Events').keys(filter_name='*_lepton*genPartFlav')
        gen_photon_branches= uproot.open(file+':Events').keys(filter_name='*_photon*genPartFlav')
        true_events = uproot.open(file)['nEventsGenWeighted'].values()[0]
        init_branches.extend(add_branches)
        init_branches.extend(gen_lepton_branches)
        init_branches.extend(gen_photon_branches)
        init_branches.append(branch)
        
    branches = uproot.open(file+':Events').arrays(init_branches, library='pd')
    
    HLT_SingleMuon = branches.loc[:,'HLT_IsoMu24'] == True
    HLT_DoubleMuon = branches.loc[:,'HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8'] == True
    HLT_EGamma = branches.loc[:,'HLT_Ele32_WPTight_Gsf'] == True
    HLT_DoubleEG = branches.loc[:,'HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL'] == True
    HLT_MuonEG1 = branches.loc[:,'HLT_Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ'] == True
    HLT_MuonEG2 = branches.loc[:,'HLT_Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL'] == True
    if 'SingleMuon' in file:
        arrays = branches.loc[HLT_SingleMuon, :].copy()
    elif 'DoubleMuon' in file:
        arrays = branches.loc[~HLT_SingleMuon & HLT_DoubleMuon, :].copy()
#         2018 is special
    elif 'EGamma' in file:
        arrays = branches.loc[~HLT_SingleMuon & ~HLT_DoubleMuon &   (HLT_EGamma | HLT_DoubleEG) ,:].copy()
    elif 'MuonEG' in file:
        arrays = branches.loc[~HLT_SingleMuon & ~HLT_DoubleMuon &  ~(HLT_EGamma | HLT_DoubleEG) & (HLT_MuonEG1 | HLT_MuonEG2),:].copy()
    else:
        arrays = branches.loc[HLT_SingleMuon | HLT_DoubleMuon |  HLT_EGamma | HLT_DoubleEG | HLT_MuonEG1 | HLT_MuonEG2,:].copy()
    
    if channel == 0: 
        channel_cut = (branches.loc[:,'channel_mark'] >= 1) & (branches.loc[:,'channel_mark'] <= 4)
    elif channel == 10:
        channel_cut = (branches.loc[:,'channel_mark'] >= 11) & (branches.loc[:,'channel_mark'] <= 14)
    elif channel == 9:
        channel_cut = (branches.loc[:,'channel_mark'] >= 5) & (branches.loc[:,'channel_mark'] <= 8)
    else:
        channel_cut = branches.loc[:,'channel_mark'] == channel
    
    arrays = arrays.loc[channel_cut,:]
    
    if isData:
        for i in trange(0, len(arrays[branch]), desc=f'fill {branch} for {file}'):
            hist.Fill(float(arrays[branch].values[i]))
    else:
        arrays['Generator_weight_sgn'] = arrays['Generator_weight'].apply(lambda x: 1 if x >= 0 else -1)
        arrays['true_weight'] = lumi * xsec * 1000 * arrays['Generator_weight_sgn'] / true_events
        for i in trange(0, len(arrays[branch]), desc=f'fill {branch} for {file}'):
            hist.Fill(float(arrays[branch].values[i]), float(arrays['true_weight'].values[i]))
    
    end_time = time.time()
    print ('Time cost: %.2f\n' %(end_time-init_time))
        

In [14]:
# 0: WZG
# 1: WZG_emm
# 2: WZG_mee
# 3: WZG_eee
# 4: WZG_mmm

# 10: ttZ
# 11: ttZ_emm
# 12: ttZ_mee
# 13: ttZ_eee
# 14: ttZ_mmm

# 9: ZZ
# 5: ZZ_eemm
# 6: ZZ_mmee
# 7: ZZ_eeee
# 8: ZZ_mmmm     

channel = 5

branch = 'ZZ_mllz1'
lumi = 59.7

xbins = 5
xleft = 0
xright = 200

### Add Data 

In [15]:
filelist_data = [
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/SingleMuon_Run2018A.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/SingleMuon_Run2018B.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/SingleMuon_Run2018C.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/SingleMuon_Run2018D_0000.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/SingleMuon_Run2018D_0001.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/DoubleMuon_Run2018A.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/DoubleMuon_Run2018B.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/DoubleMuon_Run2018C.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/DoubleMuon_Run2018D_0000.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/DoubleMuon_Run2018D_0001.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/EGamma_Run2018A.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/EGamma_Run2018B.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/EGamma_Run2018C.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/EGamma_Run2018D_0000.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/EGamma_Run2018D_0001.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/MuonEG_Run2018A.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/MuonEG_Run2018B.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/MuonEG_Run2018C.root",
    "/eos/user/s/sdeng/WZG_analysis/final_skim/2018/MuonEG_Run2018D.root",
]

hist_data = ROOT.TH1F("","",xbins,xleft,xright)
hist_data.Sumw2()

In [16]:
for file in filelist_data:
    AddHist(file, hist_data, 1, 0, 0, channel, branch)

is Data


fill ZZ_mllz1 for /eos/user/s/sdeng/WZG_analysis/final_skim/2018/SingleMuon_Run2018A.root: 100%|██████████| 67/67 [00:00<00:00, 724.39it/s]


Time cost: 7.43

is Data


fill ZZ_mllz1 for /eos/user/s/sdeng/WZG_analysis/final_skim/2018/SingleMuon_Run2018B.root: 100%|██████████| 23/23 [00:00<00:00, 91180.52it/s]


Time cost: 4.50

is Data


...

Time cost: 3.72

is Data


fill ZZ_mllz1 for /eos/user/s/sdeng/WZG_analysis/final_skim/2018/MuonEG_Run2018C.root: 0it [00:00, ?it/s]


Time cost: 4.03

is Data


fill ZZ_mllz1 for /eos/user/s/sdeng/WZG_analysis/final_skim/2018/MuonEG_Run2018D.root: 0it [00:00, ?it/s]

Time cost: 2.98



### Add MC 

In [17]:
filelist_MC = {
   "TTG":
        {"name":"TTGJets", 
        "path":"/eos/user/s/sdeng/WZG_analysis/final_skim/2018/TTGJets_TuneCP5_13TeV-amcatnloFXFX-madspin-pythia8_2018.root", 
        "xsec":4.078,
        "color":3},
   "TTZ":
        {"name":"TTZToLLNuNu", 
        "path":"/eos/user/s/sdeng/WZG_analysis/final_skim/2018/TTZToLLNuNu_M-10_TuneCP5_13TeV-amcatnlo-pythia8_2018.root", 
        "xsec":0.2432,
        "color":4},
   "TTW":
        {"name":"TTWJetsToLNu", 
        "path":"/eos/user/s/sdeng/WZG_analysis/final_skim/2018/TTWJetsToLNu_TuneCP5_13TeV-amcatnloFXFX-madspin-pythia8_2018.root", 
        "xsec":0.2149,
        "color":5},
   "tZq":
        {"name":"tZq_ll", 
        "path":"/eos/user/s/sdeng/WZG_analysis/final_skim/2018/tZq_ll_4f_ckm_NLO_TuneCP5_13TeV-amcatnlo-pythia8_2018.root", 
        "xsec":0.07358,
        "color":6},
   "WWW":
        {"name":"WWW", 
        "path":"/eos/user/s/sdeng/WZG_analysis/final_skim/2018/WWW_4F_TuneCP5_13TeV-amcatnlo-pythia8_2018.root", 
        "xsec":0.2086,
        "color":7},
   "WZ":
        {"name":"WZ", 
        "path":"/eos/user/s/sdeng/WZG_analysis/final_skim/2018/WZ_TuneCP5_13TeV-pythia8_2018.root", 
        "xsec":27.6,
        "color":8},
   "ZGToLLG":
        {"name":"ZGToLLG",
        "path":"/eos/user/s/sdeng/WZG_analysis/final_skim/2018/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8_2018.root", 
        "xsec":55.48,
        "color":9},
    "ZZ":
        {"name":"ZZ",
        "path":"/eos/user/s/sdeng/WZG_analysis/final_skim/2018/ZZ_TuneCP5_13TeV-pythia8_2018.root",
        "xsec":12.14,
        "color":10},
    "WZG":
        {"name":"signal",
        "path":"/eos/user/s/sdeng/WZG_analysis/final_skim/2018/wz",
        "xsec":0.0384,
        "color":11}
}



In [29]:
def SetHistStyle(hist, color):
    hist.SetMarkerStyle(20)
    hist.SetMarkerColor(color)
    hist.SetFillColor(color)
    hist.SetYTitle('events/bin')
    hist.SetStats(0)
    
stack_mc = ROOT.THStack("","")
hist_MC_total = ROOT.TH1F("","",xbins,xleft,xright)


for file in filelist_MC:
    hist_MC_temp = ROOT.TH1F("","",xbins,xleft,xright)
    hist_MC_temp.Sumw2()
    SetHistStyle(hist_MC_temp, filelist_MC[file]["color"])
    AddHist(filelist_MC[file]["path"], hist_MC_temp, 0, filelist_MC[file]["xsec"], lumi, channel, branch)
    filelist_MC[file]["hist"] = hist_MC_temp
    stack_mc.Add(hist_MC_temp)
    hist_MC_total.Add(hist_MC_temp)
    

is MC


fill ZZ_mllz1 for /eos/user/s/sdeng/WZG_analysis/final_skim/2018/TTGJets_TuneCP5_13TeV-amcatnloFXFX-madspin-pythia8_2018.root: 0it [00:00, ?it/s]


Time cost: 7.55

is MC


fill ZZ_mllz1 for /eos/user/s/sdeng/WZG_analysis/final_skim/2018/TTZToLLNuNu_M-10_TuneCP5_13TeV-amcatnlo-pythia8_2018.root: 100%|██████████| 986/986 [00:00<00:00, 102272.27it/s]


Time cost: 7.01

is MC


...

fill ZZ_mllz1 for /eos/user/s/sdeng/WZG_analysis/final_skim/2018/ZGToLLG_01J_5f_TuneCP5_13TeV-amcatnloFXFX-pythia8_2018.root: 100%|██████████| 2/2 [00:00<00:00, 7869.24it/s]


Time cost: 6.84

is MC


fill ZZ_mllz1 for /eos/user/s/sdeng/WZG_analysis/final_skim/2018/ZZ_TuneCP5_13TeV-pythia8_2018.root: 100%|██████████| 484/484 [00:00<00:00, 101792.26it/s]

Time cost: 6.53

is MC


FileNotFoundError: file not found

    ''

Files may be specified as:
   * str/bytes: relative or absolute filesystem path or URL, without any colons
         other than Windows drive letter or URL schema.
         Examples: "rel/file.root", "C:\abs\file.root", "http://where/what.root"
   * str/bytes: same with an object-within-ROOT path, separated by a colon.
         Example: "rel/file.root:tdirectory/ttree"
   * pathlib.Path: always interpreted as a filesystem path or URL only (no
         object-within-ROOT path), regardless of whether there are any colons.
         Examples: Path("rel:/file.root"), Path("/abs/path:stuff.root")

Functions that accept many files (uproot.iterate, etc.) also allow:
   * glob syntax in str/bytes and pathlib.Path.
         Examples: Path("rel/*.root"), "/abs/*.root:tdirectory/ttree"
   * dict: keys are filesystem paths, values are objects-within-ROOT paths.
         Example: {"/data_v1/*.root": "ttree_v1", "/data_v2/*.root": "ttree_v2"}
   * already-ope

## Add FakeLepton

In [ ]:
def AddHist_FakeLepton(file, hist, isData, xsec, lumi, channel, branch):
    
    init_time = time.time()
    init_branches = ['channel_mark','HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL','HLT_Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL','HLT_Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ','HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8','HLT_Ele32_WPTight_Gsf','HLT_IsoMu24']
    
    if isData:
        print('is Data')
        init_branches.append(branch)
    else:
        print('is MC')
        add_branches = ['Generator_weight']
        gen_branches = uproot.open(file+':Events').keys(filter_name='*genPartFlav')
        true_events = uproot.open(file)['nEventsGenWeighted'].values()[0]
        init_branches.extend(add_branches)
        init_branches.extend(gen_branches)
        init_branches.append(branch)
        
    branches = uproot.open(file+':Events').arrays(init_branches, library='pd')
    
    HLT_SingleMuon = branches.loc[:,'HLT_IsoMu24'] == True
    HLT_DoubleMuon = branches.loc[:,'HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8'] == True
    HLT_EGamma = branches.loc[:,'HLT_Ele32_WPTight_Gsf'] == True
    HLT_DoubleEG = branches.loc[:,'HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL'] == True
    HLT_MuonEG1 = branches.loc[:,'HLT_Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ'] == True
    HLT_MuonEG2 = branches.loc[:,'HLT_Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL'] == True
    if 'SingleMuon' in file:
        arrays = branches.loc[HLT_SingleMuon, :].copy()
    elif 'DoubleMuon' in file:
        arrays = branches.loc[~HLT_SingleMuon & HLT_DoubleMuon, :].copy()
#         2018 is special
    elif 'EGamma' in file:
        arrays = branches.loc[~HLT_SingleMuon & ~HLT_DoubleMuon &   (HLT_EGamma | HLT_DoubleEG) ,:].copy()
    elif 'MuonEG' in file:
        arrays = branches.loc[~HLT_SingleMuon & ~HLT_DoubleMuon &  ~(HLT_EGamma | HLT_DoubleEG) & (HLT_MuonEG1 | HLT_MuonEG2),:].copy()
    else:
        arrays = branches.loc[HLT_SingleMuon | HLT_DoubleMuon |  HLT_EGamma | HLT_DoubleEG | HLT_MuonEG1 | HLT_MuonEG2,:].copy()
    
    if channel == 0: 
        channel_cut = (branches.loc[:,'channel_mark'] >= 1) & (branches.loc[:,'channel_mark'] <= 4)
    elif channel == 10:
        channel_cut = (branches.loc[:,'channel_mark'] >= 11) & (branches.loc[:,'channel_mark'] <= 14)
    elif channel == 9:
        channel_cut = (branches.loc[:,'channel_mark'] >= 5) & (branches.loc[:,'channel_mark'] <= 8)
    else:
        channel_cut = branches.loc[:,'channel_mark'] == channel
    
    arrays = arrays.loc[channel_cut,:]
    
    if isData:
        for i in trange(0, len(arrays[branch]), desc=f'fill {branch} for {file}'):
            hist.Fill(float(arrays[branch].values[i]))
    else:
        arrays['Generator_weight_sgn'] = arrays['Generator_weight'].apply(lambda x: 1 if x >= 0 else -1)
        arrays['true_weight'] = lumi * xsec * 1000 * arrays['Generator_weight_sgn'] / true_events
        for i in trange(0, len(arrays[branch]), desc=f'fill {branch} for {file}'):
            hist.Fill(float(arrays[branch].values[i]), float(arrays['true_weight'].values[i]))
    
    end_time = time.time()
    print ('Time cost: %.2f\n' %(end_time-init_time))
        

## Add FakePhoton

## Plot

In [28]:

# file = '/eos/user/s/sdeng/WZG_analysis/final_skim/2018/SingleMuon_Run2018A.root'
file = '/eos/user/s/sdeng/WZG_analysis/final_skim/2018/TTGJets_TuneCP5_13TeV-amcatnloFXFX-madspin-pythia8_2018.root'

gen_branches = uproot.open(file+':Events').keys(filter_name='*_lepton*genPartFlav')
print (gen_branches)
    
init_branches = ['channel_mark','HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL','HLT_Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL','HLT_Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ','HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8','HLT_Ele32_WPTight_Gsf','HLT_IsoMu24']
    
branch = 'ZZ_mllz1'

init_branches.append(branch)
init_branches.extend(gen_branches)
print (init_branches)
        
init_time = time.time()
branches = uproot.open(file+':Events').arrays(init_branches, library='pd')
end_time = time.time()
print ('%.2f' %(end_time-init_time))
branches
    

['ZZ_lepton1_genPartFlav', 'ZZ_lepton2_genPartFlav', 'ZZ_lepton3_genPartFlav', 'ZZ_lepton4_genPartFlav', 'WZG_lepton1_genPartFlav', 'WZG_lepton2_genPartFlav', 'WZG_lepton3_genPartFlav', 'ttZ_lepton1_genPartFlav', 'ttZ_lepton2_genPartFlav', 'ttZ_lepton3_genPartFlav']
['channel_mark', 'HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL', 'HLT_Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL', 'HLT_Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ', 'HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8', 'HLT_Ele32_WPTight_Gsf', 'HLT_IsoMu24', 'ZZ_mllz1', 'ZZ_lepton1_genPartFlav', 'ZZ_lepton2_genPartFlav', 'ZZ_lepton3_genPartFlav', 'ZZ_lepton4_genPartFlav', 'WZG_lepton1_genPartFlav', 'WZG_lepton2_genPartFlav', 'WZG_lepton3_genPartFlav', 'ttZ_lepton1_genPartFlav', 'ttZ_lepton2_genPartFlav', 'ttZ_lepton3_genPartFlav']
2.36


,channel_mark,HLT_Ele23_Ele12_CaloIdL_TrackIdL_IsoVL,HLT_Mu23_TrkIsoVVL_Ele12_CaloIdL_TrackIdL_IsoVL,HLT_Mu8_TrkIsoVVL_Ele23_CaloIdL_TrackIdL_IsoVL_DZ,HLT_Mu17_TrkIsoVVL_Mu8_TrkIsoVVL_DZ_Mass3p8,HLT_Ele32_WPTight_Gsf,HLT_IsoMu24,ZZ_mllz1,ZZ_lepton1_genPartFlav,ZZ_lepton2_genPartFlav,ZZ_lepton3_genPartFlav,ZZ_lepton4_genPartFlav,WZG_lepton1_genPartFlav,WZG_lepton2_genPartFlav,WZG_lepton3_genPartFlav,ttZ_lepton1_genPartFlav,ttZ_lepton2_genPartFlav,ttZ_lepton3_genPartFlav
0,11,False,True,True,True,False,True,0.000000,0,0,0,0,0,0,0,22,1,1
1,12,True,True,True,False,True,False,0.000000,0,0,0,0,0,0,0,5,15,15
2,13,True,False,False,False,True,False,0.000000,0,0,0,0,0,0,0,1,1,1
3,8,False,False,False,True,False,True,78.875473,1,15,1,1,0,0,0,1,1,1
4,14,False,False,False,True,False,True,78.875473,1,15,1,1,0,0,0,1,1,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,11,False,True,True,True,True,True,0.000000,0,0,0,0,0,0,0,1,1,1
240,14,False,False,False,True,False,True,0.000000,0,0,0,0,0,0,0,15,1,5
241,11,False,False,False,False,False,True,0.000000,0,0,0,0,0,0,0,0,1,1
242,13,True,False,False,False,True,False,0.000000,0,0,0,0,0,0,0,22,1,1
